In [ ]:
!mkdir .kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp ./drive/MyDrive/kaggle.json /root/.kaggle

In [ ]:
!kaggle competitions download -c dogus-datathon-otomotiv

  0% 0.00/2.69M [00:00<?, ?B/s]
100% 2.69M/2.69M [00:00<00:00, 43.9MB/s]
  0% 0.00/4.71M [00:00<?, ?B/s]
100% 4.71M/4.71M [00:00<00:00, 43.2MB/s]
  0% 0.00/525k [00:00<?, ?B/s]
100% 525k/525k [00:00<00:00, 8.43MB/s]
  0% 0.00/5.11M [00:00<?, ?B/s]
100% 5.11M/5.11M [00:00<00:00, 47.1MB/s]
  0% 0.00/7.75M [00:00<?, ?B/s]
100% 7.75M/7.75M [00:00<00:00, 71.4MB/s]
 71% 9.00M/12.6M [00:00<00:00, 60.4MB/s]
100% 12.6M/12.6M [00:00<00:00, 61.5MB/s]
 40% 5.00M/12.5M [00:00<00:00, 52.0MB/s]
100% 12.5M/12.5M [00:00<00:00, 79.7MB/s]


# Intro

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

F_CUST_DATA = pd.read_csv("FINAL_CUSTOMER_DATATHON.csv.zip") 
F_CUST_RELATED_DATA = pd.read_csv("FINAL_CUSTOMER_RELATED_TABLE_FOR_DATATHON.csv.zip") 
F_SALES_DATA = pd.read_csv("FINAL_SALES_FILE_DATATHON.csv.zip") 
F_SIFIR_ARAC_DATA = pd.read_csv("FINAL_SIFIR_ARAC_ALANLAR_DATATHON.csv.zip") 
F_VEHICLE_DATA = pd.read_csv("FINAL_VEHICLE_TABLE_DATATHON.csv.zip") 
MASK_SERVIS_DATA = pd.read_csv("MASK_SERVIS_BAKIM_DATATHON_FINAL.csv.zip") 
ALL_DATAS = np.array([F_CUST_DATA,F_CUST_RELATED_DATA,F_SALES_DATA,F_SIFIR_ARAC_DATA,F_VEHICLE_DATA,MASK_SERVIS_DATA])
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
current_year = 2021

In [ ]:
# sifir_arac_data = pd.merge(F_SIFIR_ARAC_DATA,F_VEHICLE_DATA,on = 'VEHICLE_ID')
# common = F_VEHICLE_DATA.index.intersection(F_SIFIR_ARAC_DATA.VEHICLE_ID)

# Null Value Check

In [ ]:
F_CUST_DATA.isna().sum()

Unnamed: 0                            0
BASE_CUSTOMER_ID                      0
CUSTOMER_ID                           0
GENDER                                3
GENDER_ID                             0
MARITAL_STATUS                   229117
MARITAL_STATUS_ID                     0
BIRTH_DATE                       204472
FK_ADDRESS_COMMUNICATION_CITY    119578
OCCUPATION                       468867
dtype: int64

In [ ]:
F_CUST_RELATED_DATA.isna().sum()

Unnamed: 0                             0
BASE_CUSTOMER_ID                       0
VEHICLE_ID                             0
START_DATE                             0
END_DATE                          215122
FK_RELATION_STATUS_ID                  0
FK_RELATION_STATUS_EXPLANATION         0
dtype: int64

In [ ]:
F_SALES_DATA.isna().sum()

Unnamed: 0             0
CUSTOMER_ID            0
SALESFILE_ID           0
SF_CREATE_DATE         0
STATUS                 0
REQ_BRAND_CODE         0
REQ_TOPMODEL_CODE    289
dtype: int64

In [ ]:
F_SIFIR_ARAC_DATA.isna().sum()

Unnamed: 0     0
VEHICLE_ID     0
CUSTOMER_ID    0
CREATE_DATE    0
dtype: int64

In [ ]:
F_VEHICLE_DATA.isna().sum()

Unnamed: 0            0
VEHICLE_ID            0
TRAFFIC_DATE        866
BRAND_CODE            0
BASEMODEL_CODE        0
TOPMODEL_CODE         0
MOTOR_GAS_TYPE    72446
GEAR_BOX_TYPE     84015
dtype: int64

In [ ]:
F_VEHICLE_DATA.shape[0]

345092

# Dealing With Null Values

### fill_the_table_with_baseid

In [ ]:

def fill_the_table_with_baseid(df,base_column, report_in_every=10000):
  filled_count = 0 
  count = 0

  for column in df: # iterate through columns
    count=0
    if column == base_column: # skip the base column
      continue
    df_null = df[ df[column].isna() ] # take the null values of our column

    for index in df_null.index: # iterate through null values of our column
      # report the filled row percentage in every 10,000 rows.
      if count % report_in_every == 0: 
        filled_percentage = (count/df_null.shape[0]) * 100
        print(column, filled_percentage)
      BASE_ID = df.loc[index, base_column]
      df_BASEID = df[ df[base_column]==BASE_ID ] 

      df_non_empty = df_BASEID[ df_BASEID[column].notna() ]
      #get non na values of a specific baseid
      if df_non_empty.shape[0] > 0: # if baseid include non na value then::
        df.at[index, column] = df_non_empty[column].iloc[0] #
        filled_count += 1
      # for row_index, row in df_BASEID.iterrows():
      #   if not pd.isna(row[column]): # if this value is not na, then use its value
      #     new_value = row[column]
      #     df.at[index, column] = new_value
      #     filled_count += 1
      #     break
      count += 1
    print(column, 100.0)
  print('Filled', filled_count, 'values in total.')
  return df

### Selimin Kodu

In [ ]:
#%%
filled_customer['OCCUPATION'] = filled_customer['OCCUPATION'].fillna("Diğer");
filled_customer['BIRTH_DATE'].fillna(filled_customer['BIRTH_DATE'].median(),inplace=True)
# yüzde 82 evli 
filled_customer['MARITAL_STATUS'].value_counts(normalize=True)
filled_customer['MARITAL_STATUS'].fillna("Evli",inplace=True)
filled_customer.dropna(inplace=True)

# Customer DF Operations

In [ ]:
df_customer = F_CUST_DATA.copy()

In [ ]:
df_customer = df_customer.drop('MARITAL_STATUS_ID', axis=1)
df_customer = df_customer.drop('GENDER_ID', axis=1)

In [ ]:
df_customer.isna().sum()

Unnamed: 0                            0
BASE_CUSTOMER_ID                      0
CUSTOMER_ID                           0
GENDER                                3
MARITAL_STATUS                   229117
BIRTH_DATE                       204472
FK_ADDRESS_COMMUNICATION_CITY    119578
OCCUPATION                       468867
dtype: int64

In [ ]:
df_customer.dtypes

Unnamed: 0                         int64
BASE_CUSTOMER_ID                   int64
CUSTOMER_ID                        int64
GENDER                            object
MARITAL_STATUS                    object
BIRTH_DATE                       float64
FK_ADDRESS_COMMUNICATION_CITY     object
OCCUPATION                        object
dtype: object

In [ ]:
df_customer.dropna().agg([min, max])

,Unnamed: 0,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,BIRTH_DATE,FK_ADDRESS_COMMUNICATION_CITY,OCCUPATION
min,1,3,1000003,Erkek,Bekar,1914.0,Adana,Akademisyen / Öğr.görevlisi / Araştırma Görevlisi
max,849918,196556,1849873,Kadın,Evli,2020.0,Şırnak,Şöför / Taksici / Transfer


In [ ]:
df_customer['BASE_CUSTOMER_ID'] = df_customer['BASE_CUSTOMER_ID'].astype('uint32')
df_customer['CUSTOMER_ID'] = df_customer['CUSTOMER_ID'].astype('uint32')
df_customer['Unnamed: 0'] = df_customer['Unnamed: 0'].astype('uint32')
df_customer.dtypes

Unnamed: 0                        uint32
BASE_CUSTOMER_ID                  uint32
CUSTOMER_ID                       uint32
GENDER                            object
MARITAL_STATUS                    object
BIRTH_DATE                       float64
FK_ADDRESS_COMMUNICATION_CITY     object
OCCUPATION                        object
dtype: object

## CUST_DATA Filling The Customer Values

### Old Code

In [ ]:
# # count = 0
# # filled_count = 0

# for column in df:
#   # count=0
#   if column == 'BASE_CUSTOMER_ID':
#     continue
#   df_null = df[ df[column].isna() ]
#   for index in df_null.index:
#     # if count % 50000 == 0:
#     #   print(column, count)
#     base_customer = df.loc[index, 'BASE_CUSTOMER_ID']
#     df_customer = df[ df['BASE_CUSTOMER_ID']==base_customer ] 
#     for row_index, row in df_customer.iterrows():
#       if not pd.isna(row)[column]:
#         new_value = row[column]
#         df.at[index, column] = new_value
#         # print(column, index, new_value)
#         # filled_count += 1
#         break
#     # count += 1
# # print(filled_count)

MARITAL_STATUS 0
MARITAL_STATUS 50000
MARITAL_STATUS 100000
MARITAL_STATUS 150000
MARITAL_STATUS 200000
BIRTH_DATE 0
BIRTH_DATE 50000
BIRTH_DATE 100000
BIRTH_DATE 150000
BIRTH_DATE 200000
FK_ADDRESS_COMMUNICATION_CITY 0
FK_ADDRESS_COMMUNICATION_CITY 50000
FK_ADDRESS_COMMUNICATION_CITY 100000
OCCUPATION 0
OCCUPATION 50000
OCCUPATION 100000
OCCUPATION 150000
OCCUPATION 200000
OCCUPATION 250000
OCCUPATION 300000
OCCUPATION 350000
OCCUPATION 400000
OCCUPATION 450000
638398


### New Code

In [ ]:
fill_the_table_with_baseid(df_customer, 'BASE_CUSTOMER_ID', report_in_every=80000)

Unnamed: 0 100.0
CUSTOMER_ID 100.0
GENDER 0.0
GENDER 100.0
MARITAL_STATUS 0.0
MARITAL_STATUS 91.41498977294802
MARITAL_STATUS 100.0
BIRTH_DATE 0.0
BIRTH_DATE 39.12516139129074
BIRTH_DATE 78.25032278258148
BIRTH_DATE 100.0
FK_ADDRESS_COMMUNICATION_CITY 0.0
FK_ADDRESS_COMMUNICATION_CITY 66.90193848366756
FK_ADDRESS_COMMUNICATION_CITY 100.0
OCCUPATION 0.0
OCCUPATION 17.062407889657408
OCCUPATION 34.124815779314815
OCCUPATION 51.18722366897222
OCCUPATION 68.24963155862963
OCCUPATION 85.31203944828704
OCCUPATION 100.0
Filled 496794 values in total.


,Unnamed: 0,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,BIRTH_DATE,FK_ADDRESS_COMMUNICATION_CITY,OCCUPATION
0,0,158891,1747700,Erkek,Evli,1975.0,İstanbul,Diğer
1,1,169233,1710561,Erkek,Evli,1962.0,Erzurum,Öğretmen / Eğitmen
2,2,30887,1043923,Erkek,Evli,1971.0,İstanbul,Esnaf
3,3,38013,1101926,Erkek,NaN,NaN,Manisa,NaN
4,4,157091,1819787,Kadın,Evli,1949.0,Ankara,Öğretmen / Eğitmen
...,...,...,...,...,...,...,...,...
849914,849914,18175,1744051,Erkek,Evli,1984.0,Gaziantep,Tekstilci
849915,849915,108604,1209486,Erkek,Evli,1969.0,İstanbul,Nakliyeci / Lojistik / Taşımacılık
849916,849916,155926,1670812,Erkek,Evli,1970.0,İstanbul,NaN
849917,849917,123223,1691689,Erkek,Evli,1970.0,Kayseri,NaN


In [ ]:
df_customer.to_csv('df_customer_filled_v2.csv')

In [ ]:
!cp df_customer_filled_v2.csv drive/MyDrive/

In [51]:
df_customer = pd.read_csv('drive/MyDrive/df_customer_filled_v2.csv')

## Customer Drop City

In [52]:
df_customer = df_customer.drop('FK_ADDRESS_COMMUNICATION_CITY', axis=1)

In [68]:
df_customer.head(1)

,Unnamed: 0,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,BIRTH_DATE,OCCUPATION
0,0,158891,1747700,Erkek,Evli,1975.0,Diğer


## Customer BIRTH_DATE to AGE

In [70]:
df_customer['AGE'] = df_customer.BIRTH_DATE.map(lambda date: current_year-date)
df_customer.head(5)

,Unnamed: 0,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,BIRTH_DATE,OCCUPATION,AGE
0,0,158891,1747700,Erkek,Evli,1975.0,Diğer,46.0
1,1,169233,1710561,Erkek,Evli,1962.0,Öğretmen / Eğitmen,59.0
2,2,30887,1043923,Erkek,Evli,1971.0,Esnaf,50.0
3,3,38013,1101926,Erkek,NaN,NaN,NaN,NaN
4,4,157091,1819787,Kadın,Evli,1949.0,Öğretmen / Eğitmen,72.0


In [71]:
df_customer = df_customer.drop('BIRTH_DATE', axis=1)
df_customer.head(1)

,Unnamed: 0,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,OCCUPATION,AGE
0,0,158891,1747700,Erkek,Evli,Diğer,46.0


In [73]:
df_customer.drop(df_customer.columns[0], axis=1).to_csv('drive/MyDrive/df_customer_filled_v2.csv')

In [74]:
df_customer.head(1)

,Unnamed: 0,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,OCCUPATION,AGE
0,0,158891,1747700,Erkek,Evli,Diğer,46.0


# VEHICLE

### Vehicle Typo Correct

In [ ]:
typo_gear = np.array([
    ('Otomatik (DSG)', 'OTOMATİK'),
    ('Otomatik', 'OTOMATİK'),
    ('Mekanik', 'MANUEL'),
    ('Sürekli Değişken', 'OTOMATİK'),
    ('Otomaik', 'OTOMATİK'),
    ('?automatisch', 'OTOMATİK'),
    ('Otomatik Değişken', 'OTOMATİK'),
    ('Sürekli değişken', 'OTOMATİK'),
    ('Otomatik (DSG Şanzuman)', 'OTOMATİK'),
    ('Düz', 'MANUEL'),
    ('Otomatik(DSG)', 'OTOMATİK'),
    ('?Handschaltung', 'MANUEL'),
    ('manuel', 'MANUEL'),
    ('Manuel', 'MANUEL')
])
typo_gas_type = np.array([
    ('Diesel', 'Dizel'),
    ('Kurşunsuz benzin', 'Benzin'),
    ('Kurşunsuz Benzin', 'Benzin'),
    ('Seçiniz', None),
    ('?Benzin', 'Benzin'),
    ('Benzin/Hybrid', 'Benzin/Hibrit'),
])

def typo_regulate(typo,column_name,data):
    for key,val in typo:
        data[column_name].replace(key,val,inplace=True)

df_vehicle = F_VEHICLE_DATA.copy()
typo_regulate(typo_gear,'GEAR_BOX_TYPE', df_vehicle)
typo_regulate(typo_gas_type,'MOTOR_GAS_TYPE', df_vehicle)

In [ ]:
df_vehicle['GEAR_BOX_TYPE'].unique()

array(['MANUEL', 'OTOMATİK', nan], dtype=object)

In [ ]:
df_vehicle.isna().sum()

Unnamed: 0            0
VEHICLE_ID            0
TRAFFIC_DATE        866
BRAND_CODE            0
BASEMODEL_CODE        0
TOPMODEL_CODE         0
MOTOR_GAS_TYPE    73526
GEAR_BOX_TYPE     84015
dtype: int64

### Vehicle Dtype Casting


In [ ]:
df_vehicle.dtypes

Unnamed: 0         int64
VEHICLE_ID         int64
TRAFFIC_DATE      object
BRAND_CODE        object
BASEMODEL_CODE     int64
TOPMODEL_CODE      int64
MOTOR_GAS_TYPE    object
GEAR_BOX_TYPE     object
dtype: object

In [ ]:
df_vehicle.agg([min,max])

,Unnamed: 0,VEHICLE_ID,BRAND_CODE,BASEMODEL_CODE,TOPMODEL_CODE
min,0,5000001,K,10706,9083
max,345091,5345092,Z,18140,10010


In [ ]:
df_vehicle['BASEMODEL_CODE'] =  df_vehicle['BASEMODEL_CODE'].astype('uint16')
df_vehicle['TOPMODEL_CODE'] = df_vehicle['TOPMODEL_CODE'].astype('uint16')
df_vehicle['VEHICLE_ID'] = df_vehicle['VEHICLE_ID'].astype('uint32')
df_vehicle['Unnamed: 0'] = df_vehicle['VEHICLE_ID'].astype('uint32')
df_vehicle.dtypes

Unnamed: 0        uint32
VEHICLE_ID        uint32
TRAFFIC_DATE      object
BRAND_CODE        object
BASEMODEL_CODE    uint16
TOPMODEL_CODE     uint16
MOTOR_GAS_TYPE    object
GEAR_BOX_TYPE     object
dtype: object

In [ ]:
F_VEHICLE_DATA

,Unnamed: 0,VEHICLE_ID,TRAFFIC_DATE,BRAND_CODE,BASEMODEL_CODE,TOPMODEL_CODE,MOTOR_GAS_TYPE,GEAR_BOX_TYPE
0,0,5317713,2017-12-07,Z,13398,9349,Benzin,MANUEL
1,1,5259212,2014-12-09,X,15694,9524,Dizel,OTOMATİK
2,2,5110414,2005-12-12,Y,11183,9118,Benzin,OTOMATİK
3,3,5101263,2005-08-23,X,15039,9495,Benzin,MANUEL
4,4,5053909,2003-04-09,Y,12255,9219,NaN,NaN
...,...,...,...,...,...,...,...,...
345087,345087,5253033,2014-07-24,T,13515,9352,Benzin,MANUEL
345088,345088,5174679,2010-05-27,T,13833,9376,Benzin,MANUEL
345089,345089,5059846,2004-12-14,X,15316,9509,Benzin,OTOMATİK
345090,345090,5080882,2004-06-29,K,14615,9446,Dizel,MANUEL


### VEHICLE_DATA Filling

In [ ]:
fill_the_table_with_baseid(df_vehicle, 'TOPMODEL_CODE')

Unnamed: 0 100.0
VEHICLE_ID 100.0
TRAFFIC_DATE 0.0
TRAFFIC_DATE 100.0
BRAND_CODE 100.0
BASEMODEL_CODE 100.0
MOTOR_GAS_TYPE 0.0
MOTOR_GAS_TYPE 13.600631069281615
MOTOR_GAS_TYPE 27.20126213856323
MOTOR_GAS_TYPE 40.80189320784484
MOTOR_GAS_TYPE 54.40252427712646
MOTOR_GAS_TYPE 68.00315534640808
MOTOR_GAS_TYPE 81.60378641568968
MOTOR_GAS_TYPE 95.2044174849713
MOTOR_GAS_TYPE 100.0
GEAR_BOX_TYPE 0.0
GEAR_BOX_TYPE 11.902636433970125
GEAR_BOX_TYPE 23.80527286794025
GEAR_BOX_TYPE 35.70790930191038
GEAR_BOX_TYPE 47.6105457358805
GEAR_BOX_TYPE 59.51318216985062
GEAR_BOX_TYPE 71.41581860382075
GEAR_BOX_TYPE 83.31845503779087
GEAR_BOX_TYPE 95.221091471761
GEAR_BOX_TYPE 100.0
Filled 64576 values in total.


,Unnamed: 0,VEHICLE_ID,TRAFFIC_DATE,BRAND_CODE,BASEMODEL_CODE,TOPMODEL_CODE,MOTOR_GAS_TYPE,GEAR_BOX_TYPE
0,5317713,5317713,2017-12-07,Z,13398,9349,Benzin,MANUEL
1,5259212,5259212,2014-12-09,X,15694,9524,Dizel,OTOMATİK
2,5110414,5110414,2005-12-12,Y,11183,9118,Benzin,OTOMATİK
3,5101263,5101263,2005-08-23,X,15039,9495,Benzin,MANUEL
4,5053909,5053909,2003-04-09,Y,12255,9219,Dizel,OTOMATİK
...,...,...,...,...,...,...,...,...
345087,5253033,5253033,2014-07-24,T,13515,9352,Benzin,MANUEL
345088,5174679,5174679,2010-05-27,T,13833,9376,Benzin,MANUEL
345089,5059846,5059846,2004-12-14,X,15316,9509,Benzin,OTOMATİK
345090,5080882,5080882,2004-06-29,K,14615,9446,Dizel,MANUEL


In [ ]:
# df_vehicle.to_csv('df_vehicle_filled.csv')
# !cp df_vehicle_filled.csv drive/MyDrive/

In [ ]:
# df_vehicle = pd.read_csv('drive/MyDrive/df_vehicle_filled.csv')

## VEHICLE_DATA DateTime Conversion

In [ ]:
# df_vehicle

In [ ]:
df_vehicle.isna().mean() * 100 # Get Null Values as ''Percentage''

Unnamed: 0         0.000000
Unnamed: 0.1       0.000000
VEHICLE_ID         0.000000
TRAFFIC_DATE       0.000000
BRAND_CODE         0.000000
BASEMODEL_CODE     0.000000
TOPMODEL_CODE      0.000000
MOTOR_GAS_TYPE    12.156758
GEAR_BOX_TYPE     15.033382
dtype: float64

In [ ]:
df_vehicle['TRAFFIC_DATE'] = pd.to_datetime(df_vehicle['TRAFFIC_DATE'])
df_vehicle['TRAFFIC_YEAR'] = df_vehicle['TRAFFIC_DATE'].dt.year

# df_vehicle['TRAFFIC_YEAR'].fillna(df_vehicle['TRAFFIC_YEAR'].median(), inplace=True)
# df_vehicle.isna().mean() * 100

In [ ]:
df_vehicle.head(4)

,Unnamed: 0,Unnamed: 0.1,VEHICLE_ID,TRAFFIC_DATE,BRAND_CODE,BASEMODEL_CODE,TOPMODEL_CODE,MOTOR_GAS_TYPE,GEAR_BOX_TYPE,TRAFFIC_YEAR
0,0,5317713,5317713,2017-12-07,Z,13398,9349,Benzin,MANUEL,2017
1,1,5259212,5259212,2014-12-09,X,15694,9524,Dizel,OTOMATİK,2014
2,2,5110414,5110414,2005-12-12,Y,11183,9118,Benzin,OTOMATİK,2005
3,3,5101263,5101263,2005-08-23,X,15039,9495,Benzin,MANUEL,2005


### Convert Traffic_Year to Age

In [ ]:
# for index, row in df_vehicle.iterrows():
#   year_value = row['TRAFFIC_YEAR']
#   df_vehicle['AGE'].at[index] = current_year - year_value #current_year = 2021


df_vehicle['AGE'] = df_vehicle['TRAFFIC_YEAR'].map(lambda year: (current_year-year) )

NameError: ignored

In [ ]:
current_year

2021

In [ ]:
df_vehicle.head(10)

,Unnamed: 0,Unnamed: 0.1,VEHICLE_ID,TRAFFIC_DATE,BRAND_CODE,BASEMODEL_CODE,TOPMODEL_CODE,MOTOR_GAS_TYPE,GEAR_BOX_TYPE,TRAFFIC_YEAR,AGE
0,0,5317713,5317713,2017-12-07,Z,13398,9349,Benzin,MANUEL,2017,0
1,1,5259212,5259212,2014-12-09,X,15694,9524,Dizel,OTOMATİK,2014,0
2,2,5110414,5110414,2005-12-12,Y,11183,9118,Benzin,OTOMATİK,2005,0
3,3,5101263,5101263,2005-08-23,X,15039,9495,Benzin,MANUEL,2005,0
4,4,5053909,5053909,2003-04-09,Y,12255,9219,Dizel,OTOMATİK,2003,0
5,5,5002457,5002457,2000-12-31,X,14927,9491,NaN,NaN,2000,0
6,6,5069112,5069112,2004-07-20,Y,11201,9118,Benzin,MANUEL,2004,0
7,7,5149542,5149542,2008-01-07,T,13927,9384,Dizel,MANUEL,2008,0
8,8,5344597,5344597,2021-03-16,Z,12652,9245,Benzin,MANUEL,2021,0
9,9,5117495,5117495,2006-03-28,Y,11251,9119,Benzin,MANUEL,2006,0


In [ ]:
F_VEHICLE_DATA = F_VEHICLE_DATA.drop('TRAFFIC_DATE', axis=1)

### Deal with Unique Values

In [ ]:
df_vehicle['GEAR_BOX_TYPE'].unique()

array(['Manuel', 'Otomatik (DSG)', 'Otomatik', nan, 'Mekanik',
       'Sürekli Değişken', 'Otomaik', '?automatisch', 'Otomatik Değişken',
       'Sürekli değişken', 'Otomatik (DSG Şanzuman)', 'Düz',
       'Otomatik(DSG)', '?Handschaltung', 'manuel'], dtype=object)

# CUST_RELATED_DATA Processing

In [28]:
df_cust_rel = F_CUST_RELATED_DATA.drop('FK_RELATION_STATUS_EXPLANATION', axis=1)

df_cust_rel['START_DATE'] = pd.to_datetime(df_cust_rel['START_DATE'])
df_cust_rel['END_DATE'] = pd.to_datetime(df_cust_rel['END_DATE'])

df_cust_rel['START_YEAR'] = df_cust_rel['START_DATE'].dt.year
df_cust_rel['END_YEAR'] = df_cust_rel['END_DATE'].dt.year

df_cust_rel = df_cust_rel.drop('END_DATE', axis=1)

df_cust_rel.head(3)

,Unnamed: 0,BASE_CUSTOMER_ID,VEHICLE_ID,START_DATE,FK_RELATION_STATUS_ID,START_YEAR,END_YEAR
0,0,189537,5321560,2018-07-24,2,2018,2019.0
1,1,110802,5329278,2019-07-16,1,2019,NaN
2,2,160615,5329282,2019-09-12,1,2019,NaN


In [29]:
for index, row in df_cust_rel.iterrows():
  if row.FK_RELATION_STATUS_ID == 2:
    df_cust_rel.at[index, 'TOTAL_LICENCE_TIME'] = row.END_YEAR - row.START_YEAR
  if row.FK_RELATION_STATUS_ID == 1:
    df_cust_rel.at[index,'TOTAL_LICENCE_TIME'] = current_year - row.START_YEAR 

In [32]:
df_cust_rel.head(2)

,Unnamed: 0,BASE_CUSTOMER_ID,VEHICLE_ID,START_DATE,FK_RELATION_STATUS_ID,START_YEAR,END_YEAR,TOTAL_LICENCE_TIME
0,0,189537,5321560,2018-07-24,2,2018,2019.0,1.0
1,1,110802,5329278,2019-07-16,1,2019,NaN,2.0


In [33]:
df_cust_rel = df_cust_rel.drop(['START_YEAR', 'END_YEAR'], axis=1)
df_cust_rel.head(2)

,Unnamed: 0,BASE_CUSTOMER_ID,VEHICLE_ID,START_DATE,FK_RELATION_STATUS_ID,TOTAL_LICENCE_TIME
0,0,189537,5321560,2018-07-24,2,1.0
1,1,110802,5329278,2019-07-16,1,2.0


In [81]:
df_cust_rel.drop( df_cust_rel.columns[0], axis=1).to_csv('drive/MyDrive/df_cust_rel.csv')

# SALES_DATA Processing

In [ ]:
df_sales = F_SALES_DATA
df_sales.head(2)

,Unnamed: 0,CUSTOMER_ID,SALESFILE_ID,SF_CREATE_DATE,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE,CREATE_YEAR,CREATE_MONTH
0,0,1712792,7274467.0,2017-11-17 12:15:39,101,Y,9231.0,2017,11
1,1,1149729,4839430.0,2015-11-11 17:03:28,102,K,9184.0,2015,11


In [ ]:
df_sales.nunique()

Unnamed: 0           397526
CUSTOMER_ID          183523
SALESFILE_ID         396278
SF_CREATE_DATE       395534
STATUS                    6
REQ_BRAND_CODE            5
REQ_TOPMODEL_CODE       277
dtype: int64

## Sales Date Conversion

In [ ]:
df_sales['SF_CREATE_DATE'] = pd.to_datetime(df_sales['SF_CREATE_DATE'])
df_sales['CREATE_YEAR'] = df_sales['SF_CREATE_DATE'].dt.year
df_sales['CREATE_MONTH'] = df_sales['SF_CREATE_DATE'].dt.month
df_sales.head(2)

,Unnamed: 0,CUSTOMER_ID,SALESFILE_ID,SF_CREATE_DATE,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE,CREATE_YEAR,CREATE_MONTH
0,0,1712792,7274467.0,2017-11-17 12:15:39,101,Y,9231.0,2017,11
1,1,1149729,4839430.0,2015-11-11 17:03:28,102,K,9184.0,2015,11


In [ ]:
df_sales = df_sales.drop('SF_CREATE_DATE', axis=1)

In [ ]:
df_sales.head(1)

,Unnamed: 0,CUSTOMER_ID,SALESFILE_ID,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE,CREATE_YEAR,CREATE_MONTH
0,0,1712792,7274467.0,101,Y,9231.0,2017,11


# FINAL PROCESSING

In [94]:
df_vehicle = pd.read_csv('drive/MyDrive/df_vehicle_filled.csv', index_col=0)
df_customer = pd.read_csv('drive/MyDrive/df_customer_filled_v2.csv', index_col=0)
df_cust_rel = pd.read_csv('drive/MyDrive/df_cust_rel.csv', index_col=0)

In [76]:
df_customer.nunique()

BASE_CUSTOMER_ID    196564
CUSTOMER_ID         849919
GENDER                   2
MARITAL_STATUS           2
OCCUPATION              63
AGE                    107
dtype: int64

In [79]:
df_vehicle.nunique()

Unnamed: 0.1      345092
VEHICLE_ID        345092
TRAFFIC_DATE        7428
BRAND_CODE             5
BASEMODEL_CODE      2477
TOPMODEL_CODE        346
MOTOR_GAS_TYPE         4
GEAR_BOX_TYPE          2
dtype: int64

In [80]:
df_cust_rel.nunique()

Unnamed: 0.1             374879
BASE_CUSTOMER_ID         196538
VEHICLE_ID               345092
START_DATE                 8132
FK_RELATION_STATUS_ID         2
TOTAL_LICENCE_TIME           31
dtype: int64

## Check For Null Values (as percentage)

In [62]:
df_customer.isna().mean()*100

Unnamed: 0           0.000000
BASE_CUSTOMER_ID     0.000000
CUSTOMER_ID          0.000000
GENDER               0.000235
MARITAL_STATUS       9.419839
BIRTH_DATE           6.789000
OCCUPATION          28.400589
dtype: float64

In [63]:
df_cust_rel.isna().mean()*100

Unnamed: 0               0.0
BASE_CUSTOMER_ID         0.0
VEHICLE_ID               0.0
START_DATE               0.0
FK_RELATION_STATUS_ID    0.0
TOTAL_LICENCE_TIME       0.0
dtype: float64

In [66]:
df_vehicle.isna().mean()*100

Unnamed: 0         0.000000
Unnamed: 0.1       0.000000
VEHICLE_ID         0.000000
TRAFFIC_DATE       0.000000
BRAND_CODE         0.000000
BASEMODEL_CODE     0.000000
TOPMODEL_CODE      0.000000
MOTOR_GAS_TYPE    12.156758
GEAR_BOX_TYPE     15.033382
dtype: float64

In [64]:
F_SALES_DATA.isna().mean()*100

Unnamed: 0           0.0000
CUSTOMER_ID          0.0000
SALESFILE_ID         0.0000
SF_CREATE_DATE       0.0000
STATUS               0.0000
REQ_BRAND_CODE       0.0000
REQ_TOPMODEL_CODE    0.0727
dtype: float64

In [65]:
F_SIFIR_ARAC_DATA.isna().mean()*100

Unnamed: 0     0.0
VEHICLE_ID     0.0
CUSTOMER_ID    0.0
CREATE_DATE    0.0
dtype: float64

## Fill the Null Values

### Fill CUSTOMER Data

Drop the OCCUPATION from CUSTOMER since 28% of it are null.

In [95]:
df_customer = df_customer.drop('OCCUPATION', axis=1)

In [96]:
df_customer.head(1)

,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,AGE
0,158891,1747700,Erkek,Evli,46.0


In [102]:
df_customer.dropna(subset=['GENDER'], inplace=True)
df_customer.isna().mean()*100

BASE_CUSTOMER_ID    0.000000
CUSTOMER_ID         0.000000
GENDER              0.000000
MARITAL_STATUS      9.419626
AGE                 6.788781
dtype: float64

In [104]:
cust_age_median = df_customer['AGE'].median()
cust_age_median

52.0

In [106]:
df_customer['AGE'] = df_customer['AGE'].fillna( cust_age_median )
df_customer.isna().mean()*100

BASE_CUSTOMER_ID    0.000000
CUSTOMER_ID         0.000000
GENDER              0.000000
MARITAL_STATUS      9.419626
AGE                 0.000000
dtype: float64

In [108]:
df_customer.dtypes

BASE_CUSTOMER_ID      int64
CUSTOMER_ID           int64
GENDER               object
MARITAL_STATUS       object
AGE                 float64
dtype: object

In [110]:
df_customer['AGE'] = df_customer.AGE.astype('uint16')

In [111]:
df_customer.dtypes

BASE_CUSTOMER_ID     int64
CUSTOMER_ID          int64
GENDER              object
MARITAL_STATUS      object
AGE                 uint16
dtype: object

In [112]:
df_customer.head(1)

,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,AGE
0,158891,1747700,Erkek,Evli,46


In [115]:
df_customer.isna().mean()*100

BASE_CUSTOMER_ID    0.000000
CUSTOMER_ID         0.000000
GENDER              0.000000
MARITAL_STATUS      9.419626
AGE                 0.000000
dtype: float64

In [117]:
df_customer.dropna(subset=['MARITAL_STATUS'], inplace=True)

In [118]:
df_customer.head(1)

,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,AGE
0,158891,1747700,Erkek,Evli,46


In [120]:
F_CUST_DATA.shape[0], df_customer.shape[0]

(849919, 769858)

Only about **8%** of the data are omitted from CUSTOMER table

In [ ]:
F_

### Fill 

## One Hot Encoding

### One Hot CUSTOMER

In [ ]:
# # importing OneHotEncoder
# from sklearn.preprocessing import OneHotEncoder()

In [122]:
df_customer.head(1)

,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,MARITAL_STATUS,AGE
0,158891,1747700,Erkek,Evli,46


In [124]:
df_customer_dum = pd.get_dummies(df_customer, columns=['GENDER', 'MARITAL_STATUS'])
df_customer_dum.head(5)

,BASE_CUSTOMER_ID,CUSTOMER_ID,AGE,GENDER_Erkek,GENDER_Kadın,MARITAL_STATUS_Bekar,MARITAL_STATUS_Evli
0,158891,1747700,46,1,0,0,1
1,169233,1710561,59,1,0,0,1
2,30887,1043923,50,1,0,0,1
4,157091,1819787,72,0,1,0,1
5,6590,1826742,69,1,0,0,1


### One Hot CUSTOMER_RELATED

SHOULD I ADD THIS??

In [128]:
# df_cust_rel_dum = pd.get_dummies( df_cust_rel, columns=['FK_RELATION_STATUS_ID'])
# df_cust_rel_dum.head(3)

# CREATING DATA TO FEED THE MODEL

In [133]:
# df_cust_and_cust_rel = df_customer_dum.merge(df_cust_rel, on='BASE_CUSTOMER_ID')
# df_cust_and_cust_rel.head(15)

In [ ]:
df_final_customer = df_customer_dum.copy()

for column in df_customer_dum:
  

# Exploration for Merging

In [ ]:
df_sales.head(1)

,Unnamed: 0,CUSTOMER_ID,SALESFILE_ID,SF_CREATE_DATE,STATUS,REQ_BRAND_CODE,REQ_TOPMODEL_CODE,CREATE_YEAR,CREATE_MONTH
0,0,1712792,7274467.0,2017-11-17 12:15:39,101,Y,9231.0,2017,11


In [ ]:
df_customer = F_CUST_DATA
df_customer[ df_customer['CUSTOMER_ID'] == 1712792]

,Unnamed: 0,BASE_CUSTOMER_ID,CUSTOMER_ID,GENDER,GENDER_ID,MARITAL_STATUS,MARITAL_STATUS_ID,BIRTH_DATE,FK_ADDRESS_COMMUNICATION_CITY,OCCUPATION
374633,374633,31024,1712792,Erkek,100,Evli,101,1971.0,Düzce,Diğer


In [ ]:
df_cust_rel = F_CUST_RELATED_DATA
df_cust_rel[ df_cust_rel['BASE_CUSTOMER_ID'] == 31024]
                .sort_values(by=['START_DATE'])

,Unnamed: 0,BASE_CUSTOMER_ID,VEHICLE_ID,START_DATE,END_DATE,FK_RELATION_STATUS_ID,FK_RELATION_STATUS_EXPLANATION
102956,102956,31024,5067245,2004-08-18,NaN,1,Aktif Ruhsat Sahibi
281601,281601,31024,5280177,2015-10-09,NaN,1,Aktif Ruhsat Sahibi
297993,297993,31024,5315137,2017-12-12,2020-04-30,2,Pasif Ruhsat Sahibi


In [114]:
df_vehicle = F_VEHICLE_DATA.copy()
df_vehicle[df_vehicle['VEHICLE_ID'] == 5341920]

,Unnamed: 0,VEHICLE_ID,TRAFFIC_DATE,BRAND_CODE,BASEMODEL_CODE,TOPMODEL_CODE,MOTOR_GAS_TYPE,GEAR_BOX_TYPE
127894,127894,5341920,2021-01-25,Y,16518,9625,?Benzin,?automatisch


In [ ]:
# F_SIFIR_ARAC_DATA.head(6)
df_sifir = F_SIFIR_ARAC_DATA.copy()
df_sifir[ df_sifir['VEHICLE_ID']== 5341920]

,Unnamed: 0,VEHICLE_ID,CUSTOMER_ID,CREATE_DATE
133792,133792,5341920,1573659,2021-01-22 12:26:56


In [ ]:
df_vehicle.sample(n=3) # Get 3 Random Values

,Unnamed: 0,VEHICLE_ID,TRAFFIC_DATE,BRAND_CODE,BASEMODEL_CODE,TOPMODEL_CODE,MOTOR_GAS_TYPE,GEAR_BOX_TYPE
127894,127894,5341920,2021-01-25,Y,16518,9625,?Benzin,?automatisch
47881,47881,5113901,2006-02-21,T,13907,9383,Kurşunsuz Benzin,Otomatik
164611,164611,5342339,2021-03-19,Y,16442,9605,Benzin,Otomatik (DSG)


In [ ]:
sifir_arac_data = pd.merge(df,F_VEHICLE_DATA, on='VEHICLE_ID')